# Современные библиотеки для AI-агентов

### 📌 **LangChain**
---
[Библиотека](https://www.langchain.com/)


### **LangChain** – это фреймворк для разработки приложений на основе больших языковых моделей (LLM). 

LangChain ориентирован на гибкость и широкую совместимость: поддерживаются десятки провайдеров LLM **(OpenAI, Google Gemini, AWS, Anthropic и др.)** и множество инструментов **(поиск в Интернете, базы знаний, собственные API)**. 

Библиотека нацелена на быстрое прототипирование и производство приложений – в частности, для чат-ботов, систем вопрос-ответ с помощью Retrieval-Augmented Generation (RAG), автоматизации задач и других сценариев.


<img src="./pics/langchain.png" width="75%">

In [ ]:
# pip install langchain langchain-community

In [31]:
MODEL_NAME = "mistral-nemo:12b-instruct-2407-q3_K_L"

### 1. LLM обертки
LangChain предоставляет обертки над LLM API, включая OpenAI, Anthropic, HuggingFaceHub и локальные модели (через llamacpp, Ollama, и др.).

In [ ]:
# было :) 
# from langchain_community.llms import Ollama

In [5]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model=MODEL_NAME, temperature=0.7)

response = llm.invoke("Когда изобрели парацетамол?")
print(response)

Парацетамол был впервые синтезирован в 1879 году, но его использование как лекарственного средства началось гораздо позже. В 1899 году немецкий химик Хайнрих Дрёер (Heinrich Dreser) разработал формулу парацетамола и провел первые клинические испытания.

Однако, широкое медицинское применение парацетамола началось после Второй мировой войны. В 1953 году парацетамол был утвержден для использования в Великобритании, а затем и во многих других странах мира. С тех пор парацетамол стал одним из самых распространенных обезболивающих и жаропонижающих средств, используемых во всем мире.


### 2. Prompt Templates
PromptTemplate помогает структурировать промпты и вставлять переменные.

In [6]:
from langchain.prompts import PromptTemplate

template = PromptTemplate.from_template(f"Назови {n} столиц мира.")
prompt = template.format(n=3)
print(prompt)

Назови 3 столиц мира.


In [8]:
print(llm.invoke(prompt))

1. Москва - столица России
2. Париж - столица Франции
3. Лондон - столица Великобритании


### 3. Chains (цепочки)
Chain — это композиция из шагов, включающих промпты, модели и пост обработку.

In [14]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=template)
output = chain.run(n=3)

In [15]:
print("Промпт:", template.template)
print("Ответ модели:", output)

Промпт: Назови {n} столиц мира.
Ответ модели: 1. Москва - столица России
2. Нью-Йорк - один из крупнейших городов мира, но официальная столица США находится в Вашингтоне.
3. Пекин - столица Китая


### 4. Agents

Агенты используют LLM для динамического выбора инструментов и выполнения задач.


In [21]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

wiki = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [22]:
from langchain.agents import initialize_agent, AgentType

tools = [wiki]

agent = initialize_agent(
    tools,
    llm,
    agent='zero-shot-react-description',
    verbose=True,
)

agent.run("Кто такой Петр I и чем он знаменит? Ответ нужен на русском языке.")

/var/folders/j_/mhkb8_q935160j788zvw62ym5_k29d/T/ipykernel_8416/2133148101.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
Я должен найти информацию о Петре I и узнать, чем он известен. Лучшим инструментом для этого будет Wikipedia.
Action: wikipedia
Action Input: Петр I
Observation: Page: Peter the Great
Summary: Peter I (Russian: Пётр I Алексеевич, romanized: Pyotr I Alekseyevich, IPA: [ˈpʲɵtr ɐlʲɪkˈsʲejɪvʲɪtɕ]; 9 June [O.S. 30 May] 1672
– 8 February [O.S. 28 January] 1725), better known as Peter the Great, was the Tsar of all Russia from 1682 and the first Emperor of all Russia from 1721 until his death in 1725. He reigned jointly with his half-brother Ivan V until 1696. From this year, Peter was an absolute monarch, an autocrat who remained the ultimate authority and organized a well-ordered police state.
Much of Peter's reign was consumed by lengthy wars against the Ottoman and Swedish empires. His Azov campaigns were followed by the foundation of the Russian Navy; after his victory in the Great Northern War, Russia annexed a significant portion of the eastern B

'Петр I был российским царем, который значительно способствовал модернизации и западному влиянию своей страны. Он вел войны против Османской и Шведской империй, расширил территорию России, основал российский флот, город Санкт-Петербург, ввел юлианский календарь и поощрял индустриализацию и высшее образование в стране.Peter I в основном заслуживает признания за то, что превратил Россию в крупную европейскую державу во время своего правления.'

In [11]:
# Агент с БД

import sqlite3

from langchain_community.utilities.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain


# создадим таблицу в БД для демонстрации
conn = sqlite3.connect("students.db")
cursor = conn.cursor()

cursor.executescript("""
DROP TABLE IF EXISTS students;
DROP TABLE IF EXISTS grades;

CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    group_name TEXT
);

CREATE TABLE grades (
    id INTEGER PRIMARY KEY,
    student_id INTEGER,
    subject TEXT,
    grade INTEGER,
    FOREIGN KEY(student_id) REFERENCES students(id)
);
""")

students = [
    (1, "Алексей Иванов", "Группа А"),
    (2, "Мария Смирнова", "Группа Б"),
    (3, "Игорь Кузнецов", "Группа А"),
]
grades = [
    (1, 1, "Математика", 5),
    (2, 1, "Физика", 4),
    (3, 2, "Математика", 3),
    (4, 2, "Физика", 5),
    (5, 3, "Математика", 4),
    (6, 3, "Физика", 4),
]
cursor.executemany("INSERT INTO students VALUES (?, ?, ?)", students)
cursor.executemany("INSERT INTO grades VALUES (?, ?, ?, ?)", grades)
conn.commit()


db = SQLDatabase.from_uri("sqlite:///students.db")
# цепочка для langchain, подлючаем нашу БД
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

# обращаемся к БД 
question = "Какие оценки получил Алексей Иванов по каждому предмету?"
response = db_chain.invoke(question)
print("\ Ответ:\n", response)



> Entering new SQLDatabaseChain chain...
Какие оценки получил Алексей Иванов по каждому предмету?
SQLQuery:SELECT "subject", "grade"
FROM "grades"
INNER JOIN "students" ON "grades"."student_id" = "students"."id"
WHERE "students"."name" = 'Алексей Иванов'
LIMIT 5
SQLResult: [('Математика', 5), ('Физика', 4)]
Answer:Алексей Иванов получил оценки 5 по Математике и 4 по Физике.
> Finished chain.
\ Ответ:
 {'query': 'Какие оценки получил Алексей Иванов по каждому предмету?', 'result': 'Алексей Иванов получил оценки 5 по Математике и 4 по Физике.'}


In [25]:
question = "Сколько всего студентов было на зачете?"
response = db_chain.invoke(question)
print("\ Ответ:\n", response)



> Entering new SQLDatabaseChain chain...
Сколько всего студентов было на зачете?
SQLQuery:SELECT COUNT(DISTINCT student_id) FROM grades;
SQLResult: [(3,)]
Answer:There were 3 students who took the exam.
> Finished chain.
\ Ответ:
 {'query': 'Сколько всего студентов было на зачете?', 'result': 'There were 3 students who took the exam.'}


In [26]:
question = "Какая средняя оценка по Математике среди студентов?"
response = db_chain.invoke(question)
print("\ Ответ:\n", response)



> Entering new SQLDatabaseChain chain...
Какая средняя оценка по Математике среди студентов?
SQLQuery:SELECT AVG(grade) AS average_math_grade FROM grades WHERE subject = "Математика"
SQLResult: [(4.0,)]
Answer:Средняя оценка по Математике среди студентов - 4.0
> Finished chain.
\ Ответ:
 {'query': 'Какая средняя оценка по Математике среди студентов?', 'result': 'Средняя оценка по Математике среди студентов - 4.0'}


### 6. Memory (Память)

Память позволяет хранить историю диалога:

In [19]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
# для примера, но больше не рекомендуется использование памяти в таком виде 
# они переместили память в lang graph
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True 
)

response_1 = conversation.invoke({"input": "Привет, меня зовут Диана."})
print("\nОтвет 1:", response_1["response"])

response_2 = conversation.invoke({"input": "Как меня зовут?"})
print("\nОтвет 2:", response_2["response"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Привет, меня зовут Диана.
AI:

> Finished chain.

Ответ 1: Привет, Диана! Я — языковая модель. Какой у тебя прекрасный имя! Расскажи мне о себе, если захочешь.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Привет, меня зовут Диана.
AI: Привет, Диана! Я — языковая модель. Какой у тебя прекрасный имя! Расскажи мне о себе, если захочешь.
Human: Как меня зовут?
AI:

> Finished chain.

О

### 📌 **LangGraph**
---
[Библиотека](https://www.langchain.com/langgraph)


**LangGraph строит граф вычислений поверх модели LangChain.**

Основные сущности: 
- **StateGraph/Graph** – граф, вершины которого связаны направленными ребрами
- **Node** – узел, выполняющий функцию (Runnable или Python-функция)
- **Edge/ConditionalEdge** – переходы между узлами (обычные или условные)
- **State** – общее состояние графа (словарь), который разделяют все узлы.

В StateGraph узлы получают на вход текущее состояние и возвращают частичное обновление (например, новые значения для некоторых ключей) . 

In [ ]:
# pip install langgraph langchain langchain-community

In [37]:
# нет памяти
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph,START, END
from langgraph.graph.message import AnyMessage, add_messages
from typing import Annotated, List
from typing_extensions import TypedDict
from langchain_ollama import ChatOllama

class State(TypedDict):
    messages: Annotated[List[AnyMessage], add_messages]

llm = ChatOllama(model=MODEL_NAME)

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "{system_message}"),
        MessagesPlaceholder("messages")
    ]
)

llm_model = prompt_template | llm

graph_builder=StateGraph(State)

def ChatNode(state: State)->State:
    system_message="You are an assistant"
    state["messages"] = llm_model.invoke({"system_message": system_message, "messages": state["messages"]})
    return state

graph_builder.add_node("chatnode", ChatNode)
graph_builder.add_edge(START, "chatnode")
graph_builder.add_edge("chatnode", END)
graph = graph_builder.compile()

input_state={"messages": ["Меня зовут Диана"]}
response_state=graph.invoke(input_state)
for message in response_state["messages"]:
    message.pretty_print()


input_state={"messages":["Как меня зовут?"]}
response_state=graph.invoke(input_state)
for message in response_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Меня зовут Диана
================================== Ai Message ==================================

 Приятно познакомиться, Диана! Как я могу вам помочь сегодня?
================================ Human Message =================================

Как меня зовут?
================================== Ai Message ==================================

Вы еще не представились. Как я могу к вам обращаться?


In [38]:
# Short-Term Memory

from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph(State)

def ChatNode(state: State) -> State:
    system_message = "You are an assistant"
    state["messages"] = llm_model.invoke({"system_message": system_message, "messages": state["messages"]})
    return state

graph_builder.add_node("chatnode", ChatNode)
graph_builder.add_edge(START, "chatnode")
graph_builder.add_edge("chatnode", END)
# память
graph = graph_builder.compile(checkpointer=MemorySaver())

config= {"configurable": {"thread_id": 1}}

input_state={"messages":["Меня зовут Диана"]}
response_state=graph.invoke(input_state, config=config)
for message in response_state["messages"]:
    message.pretty_print()

input_state={"messages":["Как меня зовут?"]}
response_state=graph.invoke(input_state, config=config)
for message in response_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Меня зовут Диана
================================== Ai Message ==================================

 Приятно познакомиться, Диана! Как я могу вам помочь сегодня?
================================ Human Message =================================

Меня зовут Диана
================================== Ai Message ==================================

 Приятно познакомиться, Диана! Как я могу вам помочь сегодня?
================================ Human Message =================================

Как меня зовут?
================================== Ai Message ==================================

 Вас зовут Диана.


In [ ]:
# Теперь бот помнит моё имя
# MemorySaver сохраняет состояние, а thread_id гарантирует, что все взаимодействия будут оставаться в рамках одной цепочки сообщений. 
# Каждый новый ввод добавляется к существующей истории сообщений, предоставляя LLM полный контекст.

In [45]:
config = {"configurable": {"thread_id": 1}}
state_history = graph.get_state_history(config)
for state in state_history:
    print(state)

StateSnapshot(values={'messages': [HumanMessage(content='Меня зовут Диана', additional_kwargs={}, response_metadata={}, id='d1fee48e-a887-4091-be2c-b4a6ca275270'), AIMessage(content=' Приятно познакомиться, Диана! Как я могу вам помочь сегодня?', additional_kwargs={}, response_metadata={'model': 'mistral-nemo:12b-instruct-2407-q3_K_L', 'created_at': '2025-07-20T09:45:09.433244Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4230272958, 'load_duration': 37380500, 'prompt_eval_count': 14, 'prompt_eval_duration': 2905258375, 'eval_count': 20, 'eval_duration': 1285846417, 'model_name': 'mistral-nemo:12b-instruct-2407-q3_K_L'}, id='run--b8285167-f219-446b-95cf-77a441e1008f-0', usage_metadata={'input_tokens': 14, 'output_tokens': 20, 'total_tokens': 34}), HumanMessage(content='Как меня зовут?', additional_kwargs={}, response_metadata={}, id='ce8300a9-44fc-43f3-b62b-dc0c9876a8d3'), AIMessage(content=' Вас зовут Диана.', additional_kwargs={}, response_metadata={'model': 'mistral-nemo

In [46]:
config = {"configurable": {"thread_id": 1, "checkpoint_id":"1f0654e3-a1b3-661a-8004-92876bb9760c"}}
state = graph.get_state(config)
print(state)

StateSnapshot(values={'messages': [HumanMessage(content='Меня зовут Диана', additional_kwargs={}, response_metadata={}, id='d1fee48e-a887-4091-be2c-b4a6ca275270'), AIMessage(content=' Приятно познакомиться, Диана! Как я могу вам помочь сегодня?', additional_kwargs={}, response_metadata={'model': 'mistral-nemo:12b-instruct-2407-q3_K_L', 'created_at': '2025-07-20T09:45:09.433244Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4230272958, 'load_duration': 37380500, 'prompt_eval_count': 14, 'prompt_eval_duration': 2905258375, 'eval_count': 20, 'eval_duration': 1285846417, 'model_name': 'mistral-nemo:12b-instruct-2407-q3_K_L'}, id='run--b8285167-f219-446b-95cf-77a441e1008f-0', usage_metadata={'input_tokens': 14, 'output_tokens': 20, 'total_tokens': 34}), HumanMessage(content='Как меня зовут?', additional_kwargs={}, response_metadata={}, id='ce8300a9-44fc-43f3-b62b-dc0c9876a8d3'), AIMessage(content=' Вас зовут Диана.', additional_kwargs={}, response_metadata={'model': 'mistral-nemo

#### Но что, если вам нужно изменить состояние, например, добавить или исправить информацию? Метод update_state в LangGraph позволяет изменять состояние напрямую. Вот пример, где мы обновляем состояние, добавляя новую информацию:


In [47]:
from langchain_core.messages import HumanMessage

config = {"configurable": {"thread_id": 1}}
graph.update_state(config, {"messages": [HumanMessage(content="Я работаю исследователем данных")]})

input_state = {"messages": ["Чем я занимаюсь?"]}
response_state = graph.invoke(input_state, config=config)
for message in response_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Меня зовут Диана
================================== Ai Message ==================================

 Приятно познакомиться, Диана! Как я могу вам помочь сегодня?
================================ Human Message =================================

Как меня зовут?
================================== Ai Message ==================================

 Вас зовут Диана.
================================ Human Message =================================

Я работаю исследователем данных
================================ Human Message =================================

Чем я занимаюсь?
================================== Ai Message ==================================

 Исследованием данных. Вы работаете в качестве исследователя данных.


In [48]:
# то есть мы просто добавили новый контекст

# Вы также можете обновить состояние в определённой контрольной точке, используя checkpoint_id и продолжить выполнение оттуда. 
# Изменим последнее сообщение

config = {"configurable":{"thread_id":1}}

# все наши чекпоинты
all_checkpoints = []
for state in graph.get_state_history(config):
    all_checkpoints.append(state)

# ищем по индексу что будем менять
index = 0
selected_index = 0
for state in graph.get_state_history(config):
    index += 1
    if state.values["messages"] != [] and state.values["messages"][-1].content == "Я работаю исследователем данных":
        selected_index = index

old_config = all_checkpoints[selected_index].config
graph.update_state(old_config, {"messages": [HumanMessage(content="Я живу в Москве")]})

input_state = {"messages": ["Где я живу?"]}
response_state = graph.invoke(input_state, config=config)
for message in response_state["messages"]:
    message.pretty_print()


================================ Human Message =================================

Меня зовут Диана
================================== Ai Message ==================================

 Приятно познакомиться, Диана! Как я могу вам помочь сегодня?
================================ Human Message =================================

Как меня зовут?
================================== Ai Message ==================================

 Вас зовут Диана.
================================ Human Message =================================

Я живу в Москве
================================ Human Message =================================

Где я живу?
================================== Ai Message ==================================

 Вы живете в Москве.


In [49]:
from langgraph.graph.message import RemoveMessage

def filter_node(state: State) -> State:
    # давайте оставим только два последних сообщения
    state["messages"] = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return state

graph_builder = StateGraph(State)
graph_builder.add_node("filternode", filter_node)
graph_builder.add_node("chatnode", ChatNode)
graph_builder.add_edge(START, "filternode")
graph_builder.add_edge("filternode", "chatnode")
graph_builder.add_edge("chatnode", END)
graph = graph_builder.compile(checkpointer=MemorySaver())

In [50]:
# Теперь старые сообщения удаляются, сохраняя контекст управляемым. 
# В качестве альтернативы, вы можете использовать функцию trim_messages из LangChain для обрезки по количеству токенов.

from langchain_core.messages import trim_messages

graph_builder = StateGraph(State)

def chat_node(state: State) -> State:
    system_message = "You are an assistant"
    trimmed_messages = trim_messages(
        state["messages"],
        strategy="last",
        token_counter=llm,
        max_tokens=50,
        start_on="human",
        end_on=("human", "tool"),
        include_system=True
    )
    state["messages"] = llm_model.invoke({"system_message": system_message, "messages": trimmed_messages})
    return state

graph_builder.add_node("chatnode", ChatNode)
graph_builder.add_edge(START, "chatnode")
graph_builder.add_edge("chatnode", END)
graph = graph_builder.compile(checkpointer=MemorySaver())

In [51]:
# есть еще память, которая саммаризирует контекст, а не хранит его полностью

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import AnyMessage, add_messages, RemoveMessage
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from typing import Annotated, List, Literal
from typing_extensions import TypedDict

class State(TypedDict):
    messages:Annotated[List[AnyMessage],add_messages]
    summary: str

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "{system_message}"),
        MessagesPlaceholder("messages")
    ]
)
llm_model = prompt_template | llm

graph_builder=StateGraph(State)

def chat_node(state:State)->State:
    system_message="You are an assistant"
    summary = state.get("summary", "")
    if summary:
        system_message += f"Summary of conversation earlier: {summary}"
    state["messages"]=llm_model.invoke({"system_message":system_message,"messages":state["messages"]})
    return state

# реализуем это сами
def summarize_conversation(state: State):
    system_message="You are an chat summarizer"
    summary = state.get("summary", "")
    if summary:
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
    else:
        summary_message = "Create a summary of the conversation above:"
    response=llm_model.invoke({"system_message":system_message,"messages":state["messages"]+[HumanMessage(content=summary_message)]})
    # не храним контекст, оставляем только последних два сообщения
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

def should_continue(state: State) -> Literal["summarize", END]:
    """Return the next node to execute."""
    messages = state["messages"]
    # как только у нас больше 6 сообщений, саммаризируем его 
    if len(messages) > 6:
        return "summarize"
    return END

graph_builder.add_node("chatnode",chat_node)
graph_builder.add_node("summarize",summarize_conversation)
graph_builder.add_edge(START,"chatnode")
graph_builder.add_conditional_edges("chatnode",should_continue)
graph_builder.add_edge("summarize",END)
graph=graph_builder.compile(checkpointer=MemorySaver())

config={"configurable":{"thread_id":1}}

input_state={"messages":["Меня зовут Диана"]}
response_state=graph.invoke(input_state,config=config)
for message in response_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Меня зовут Диана
================================== Ai Message ==================================

 Приятно познакомиться, Диана! Как я могу вам помочь сегодня?


In [52]:
input_state={"messages":["Как меня зовут?"]}
response_state=graph.invoke(input_state,config=config)
for message in response_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Меня зовут Диана
================================== Ai Message ==================================

 Приятно познакомиться, Диана! Как я могу вам помочь сегодня?
================================ Human Message =================================

Как меня зовут?
================================== Ai Message ==================================

 Вас зовут Диана.


In [53]:
input_state={"messages":["Я работаю исследователем данных"]}
response_state=graph.invoke(input_state,config=config)
for message in response_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Меня зовут Диана
================================== Ai Message ==================================

 Приятно познакомиться, Диана! Как я могу вам помочь сегодня?
================================ Human Message =================================

Как меня зовут?
================================== Ai Message ==================================

 Вас зовут Диана.
================================ Human Message =================================

Я работаю исследователем данных
================================== Ai Message ==================================

 Круто! Чем вы занимаетесь в качестве исследователя данных? Какие типы данных вы изучаете и что пытаетесь выяснить с их помощью?


In [56]:
input_state={"messages":["Кем я работаю и как меня зовут??"]}
response_state=graph.invoke(input_state,config=config)
for message in response_state["messages"]:
    message.pretty_print()

================================ Human Message =================================

Кем я работаю?
================================== Ai Message ==================================

 Вы сказали, что являетесь исследователем данных.
================================ Human Message =================================

Кем я работаю и как меня зовут??
================================== Ai Message ==================================

 Я — исследователь данных, и меня зовут Диана.


### 📌 **AutoGen**
---
[Библиотека](https://microsoft.github.io/autogen/stable//index.html)


**AutoGen** – это фреймворк от Microsoft Research для создания многопроцессных (multiagent) AI-приложений . Он предназначен для разработки агентных систем, где несколько LLMагентов могут общаться друг с другом или с людьми для решения задач. AutoGen реализует асинхронную событийно-ориентированную архитектуру: агенты обмениваются сообщениями в разных режимах (запрос/ответ, события). 

Благодаря этому можно строить детерминированные и динамические бизнес-процессы с поддержкой совместной работы агентов. 


**В отличие от LangChain, AutoGen предоставляет асинхронные, событийные и диалоговые LLM-сценарии, где агенты ведут переписку, передавая сообщения и делая паузы для обработки.**


<img src="./pics/assistant-agent.svg" width="75%">

##### Есть основной "разговорный" агент ConversableAgent и его подклассы:


- [AssistantAgent](https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/assistant_agent/#assistantagent) – это ассистент ИИ, который использует LLM для генерации ответов и может вызывать инструменты и функции. 


- [UserProxyAgent](https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/user_proxy_agent)– это агент-прокси для человека: по умолчанию запрашивает ввод пользователя, но при наличии блоков кода может выполнять их автоматически.


#### Агенты общаются посредством AgentRuntime – централизованного обмена сообщениями, реализованного по принципам акторной модели. 

AutoGen поддерживает два режима выполнения: 
- **StandaloneRuntime (однопроцессный)**

<img src="/Users/dapavlik10/Downloads/llm_agents_chatbot/pics/architecture-standalone.svg" width="35%">

- **Distributed Runtime (с несколькими хостами/воркерами)**

<img src="/Users/dapavlik10/Downloads/llm_agents_chatbot/pics/architecture-distributed.svg" width="35%">


Таким образом, «протокол» взаимодействия – это асинхронный обмен объектами сообщений (текстами, объектами типа FunctionCall , ToolExecutionResult и пр.) через шину рантайма.

## Шаблоны промптов и системное сообщение в AutoGen
Каждый агент имеет конфигурацию системного сообщения (system prompt/message), которое задаётся при создании агента или берётся по умолчанию.

In [ ]:
# Внутри AssistantAgent: 

system_message: (
            str | None
        ) = """You are a helpful AI assistant. Solve tasks using your tools. 
            Reply with TERMINATE when the task has been completed."""

# Create an AssistantAgent instance that uses the tool and model client.
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[tool],
    system_message="Use the tool to analyze sentiment.",
    output_content_type=AgentResponse)

# Create an AssistantAgent instance with the model client and context.
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    model_context=model_context,
    system_message="You are a helpful assistant.",
)

In [30]:
# Внутри CodeExecutorAgent: 
# It is recommended that the `CodeExecutorAgent` agent uses a Docker container to execute code. 
# This ensures that model-generated code is executed in an isolated environment. 
# To use Docker, your environment must have Docker installed and running.

DEFAULT_SYSTEM_MESSAGE = """You are a Code Execution Agent. 
Your role is to generate and execute Python code and shell scripts based on user instructions, 
ensuring correctness, efficiency, and minimal errors. Handle edge cases gracefully.
Python code should be provided in ```python code blocks, and sh shell scripts should be provided in ```sh code blocks for execution."""


In [ ]:
# Переменная config_list определяет список LLM, которые Autogen использует для создания агентов. 
# Вы можете либо определить глобальный список с несколькими записями и указать, какой LLM выбрать для того или иного агента,
# либо определить несколько списков с одной записью, которая затем будет передана каждому конкретному агенту.


llm_config = {
    'model': 'mistral-nemo:12b-instruct-2407-q3_K_L',
    'base_url': 'http://localhost:11434/v1',
    'api_key': 'ollama',
    'cache_seed': None,
    'temperature': 0.2
}

In [ ]:
user_proxy.initiate_chat(
    assistant,
    message="Как меня зовут и что я изучаю?",
)

In [ ]:
from autogen import AssistantAgent, UserProxyAgent


# cоздание агентов. Это всегда AssistantAgent
analyst = AssistantAgent(
    name="Аналитик",
    system_message="Ты профессиональный аналитик данных. Генерируй код на Python",
    llm_config=llm_config
)

# по сути это пользователь
user_proxy = UserProxyAgent(
    name="Пользователь",
    human_input_mode="NEVER", 
    code_execution_config={"work_dir": "coding", "use_docker": False},
    max_consecutive_auto_reply=3
)

# здесь наш диалог, где будет работать 
user_proxy.initiate_chat(
    analyst,
    message="""Задача:
    1. Сгенерируй 1000 рандомных чисел
    2. Рассчитай среднее и стандартное отклонение"""
)

Пользователь (to Аналитик):

Задача:
    1. Сгенерируй 1000 рандомных чисел
    2. Рассчитай среднее и стандартное отклонение

--------------------------------------------------------------------------------
[autogen.oai.client: 06-29 15:57:55] {696} WARNING - Model mistral-nemo:12b-instruct-2407-q3_K_L is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Аналитик (to Пользователь):

Вот как вы можете сгенерировать 1000 случайных чисел, рассчитать их среднее и стандартное отклонение в Python:

```python
import numpy as np

# Шаг 1: Сгенерируйте 1000 рандомных чисел
random_numbers = np.random.rand(1000)

# Шаг 2: Рассчитайте среднее и стандартное отклонение
mean_value = np.mean(random_numbers)
std_deviation = np.std(random_numbers)

print("Среднее значение:", mean_value)
print("Стандартное отклонение:", std_deviation)
```

В этом коде мы используем библиотеку NumPy для генерации случайны

ChatResult(chat_id=None, chat_history=[{'content': 'Задача:\n    1. Сгенерируй 1000 рандомных чисел\n    2. Рассчитай среднее и стандартное отклонение', 'role': 'assistant', 'name': 'Пользователь'}, {'content': 'Вот как вы можете сгенерировать 1000 случайных чисел, рассчитать их среднее и стандартное отклонение в Python:\n\n```python\nimport numpy as np\n\n# Шаг 1: Сгенерируйте 1000 рандомных чисел\nrandom_numbers = np.random.rand(1000)\n\n# Шаг 2: Рассчитайте среднее и стандартное отклонение\nmean_value = np.mean(random_numbers)\nstd_deviation = np.std(random_numbers)\n\nprint("Среднее значение:", mean_value)\nprint("Стандартное отклонение:", std_deviation)\n```\n\nВ этом коде мы используем библиотеку NumPy для генерации случайных чисел и расчета статистических показателей. Функция `np.random.rand()` генерирует массив из 1000 случайных чисел в диапазоне от 0 до 1. Затем мы рассчитываем среднее значение с помощью функции `np.mean()` и стандартное отклонение с помощью функции `np.std()`

🔍 Что делает этот код
 1. UserProxyAgent — агент, имитирующий пользователя.

 2. AssistantAgent — агент, представляющий LLM, который будет отвечать.

 3. Инициация чата — пользователь задаёт вопрос, а ассистент пытается на него ответить.
 
 4. Автоматическая остановка происходит, если в ответе появится слово «конец».


### 📌 **CrewAI**
---
[Библиотека](https://www.crewai.com/)

**CrewAI** – фреймворк для построения команд автономных LLM-агентов с role-based организацией. 

#### Основные компоненты:
- **Crew (команда)** – верхнеуровневая организация, управляющая набором агентов и распределением задач
- **Agent (агент)** – специализированный участник с конкретной ролью, целью и инструментами
- **Task (задача)** – единичное задание с описанием и ожидаемым результатом
- **Process** – определяет схему выполнения задач (последовательную или иерархическую)
- **Flow** – дополнительный модуль для событийно-ориентированной оркестрации (Workflow) с возможностью условной логики и ветвлений

#### Агент может иметь менеджера (Manager Agent) при иерархическом процессе: в этом случае задаётся LLM и агент-менеджер для координации подчинённых 

In [29]:
# pip install crewai==0.30.7

In [30]:
# pip install crewai==0.134.0

In [28]:
# pip install crewai crewai-tools langchain langchain-community

In [ ]:
from crewai import Agent, Task, Crew, LLM

ollama_llm = LLM(
  model="ollama/mistral-nemo:12b-instruct-2407-q3_K_L",
  api_base="http://localhost:11434",
  )

agent = Agent(role="Генератор идей", goal="Придумать стартапы", backstory="Ты визионер.", llm=ollama_llm, verbose=True)
task = Task(description="Придумай 5 оригинальных идей для технологических стартапов. Пиши на русском языке", 
            expected_output="Список из 5 идей.", agent=agent)
crew = Crew(agents=[agent], tasks=[task], verbose=True)
result = crew.kickoff()
print("\n Идеи:\n", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: d2c5d962-dc4e-4a30-9c3c-40d5a7932f12                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Генератор идей                                                                                          │
│                                                                                                                 │
│  Task: Придумай 5 оригинальных идей для технологических стартапов. Пиши на русском языке                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Генератор идей                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  1. **Агротех стартап - "Зеленая Планета"**                                                                     │
│     - Идея: Разработка и внедрение умных систем для сельского хозяйства, которые помогут фермерам               │
│  оптимизировать использование воды, удобрений, а также предотвращать болезни растений.                          │
│     - Технологии: IoT, Machine Learning, беспроводные датчики, блокчейн для отслеживания происхождения          │
│  продуктов.                                                                                                     │
│     - Рынок: Сельскохозяйственная промышленность.                                                               │
│                                                                                                                 │
│  2. **Стартап в области здравоохранения - "Здоровый Образ"**                                                    │
│     - Идея: Создание искусственного интеллекта, который поможет диагностировать заболевания на ранних стадиях   │
│  путем анализа изображений медицинских тестов и истории болезни пациента.                                       │
│     - Технологии: Deep Learning, Computer Vision, Natural Language Processing (NLP).                            │
│     - Рынок: Медицинская промышленность.                                                                        │
│                                                                                                                 │
│  3. **Стартап в области образования - "Умный Школьник"**                                                        │
│     - Идея: Разработка персонализированной платформы для обучения, которая использует Machine Learning для      │
│  адаптации содержимого и темпа обучения к индивидуальным потребностям учащегося.                                │
│     - Технологии: Machine Learning, Personalized Content Recommendation System.                                 │
│     - Рынок: Образовательные учреждения.                                                                        │
│                                                                                                                 │
│  4. **Стартап в области мобильных приложений - "Эко-карта"**                                                    │
│     - Идея: Создание геолокационного мобильного приложения, которое поможет пользователям находить              │
│  экологически чистые продукты и услуги поблизости.                                                              │
│     - Технологии: Геолокация, Mobile App Development.                                                           │
│     - Рынок: Забота об окружающей среде.                                                                        │
│                                                                                                                 │
│  5. **Стартап в области кибербезопасности - "Защитник Данных"**                                                 │
│     - Идея: Разработка решения для предотвращения и обнаружения кибератак на основе Machine Learning, которое   │
│  будет мониторить сетевой трафик и блокировать подозрительную активность в режиме реального времени.            │
│     - Технологии: Machine Learning, Network Traffic Monitoring, Intrusion Detection System (IDS).               │
│     - Рынок: Индустрия кибербезопасности.              

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a49d80de-13cb-431f-b02c-c4958843f7bc                                                                     │
│  Agent: Генератор идей                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: d2c5d962-dc4e-4a30-9c3c-40d5a7932f12                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 1. **Агротех стартап - "Зеленая Планета"**                                                       │
│     - Идея: Разработка и внедрение умных систем для сельского хозяйства, которые помогут фермерам               │
│  оптимизировать использование воды, удобрений, а также предотвращать болезни растений.                          │
│     - Технологии: IoT, Machine Learning, беспроводные датчики, блокчейн для отслеживания происхождения          │
│  продуктов.                                                                                                     │
│     - Рынок: Сельскохозяйственная промышленность.                                                               │
│                                                                                                                 │
│  2. **Стартап в области здравоохранения - "Здоровый Образ"**                                                    │
│     - Идея: Создание искусственного интеллекта, который поможет диагностировать заболевания на ранних стадиях   │
│  путем анализа изображений медицинских тестов и истории болезни пациента.                                       │
│     - Технологии: Deep Learning, Computer Vision, Natural Language Processing (NLP).                            │
│     - Рынок: Медицинская промышленность.                                                                        │
│                                                                                                                 │
│  3. **Стартап в области образования - "Умный Школьник"**                                                        │
│     - Идея: Разработка персонализированной платформы для обучения, которая использует Machine Learning для      │
│  адаптации содержимого и темпа обучения к индивидуальным потребностям учащегося.                                │
│     - Технологии: Machine Learning, Personalized Content Recommendation System.                                 │
│     - Рынок: Образовательные учреждения.                                                                        │
│                                                                                                                 │
│  4. **Стартап в области мобильных приложений - "Эко-карта"**                                                    │
│     - Идея: Создание геолокационного мобильного приложения, которое поможет пользователям находить              │
│  экологически чистые продукты и услуги поблизости.                                                              │
│     - Технологии: Геолокация, Mobile App Development.                                                           │
│     - Рынок: Забота об окружающей среде.                                                                        │
│                                                                                                                 │
│  5. **Стартап в области кибербезопасности - "Защитник Данных"**                                                 │
│     - Идея: Разработка решения для предотвращения и обнаружения кибератак на основе Machine Learning, которое   │
│  будет мониторить сетевой трафик и блокировать подозрительную активность в режиме реального времени.            │
│     - Технологии: Machine Learning, Network Traffic Mo


🚀 Идеи:
 1. **Агротех стартап - "Зеленая Планета"**
   - Идея: Разработка и внедрение умных систем для сельского хозяйства, которые помогут фермерам оптимизировать использование воды, удобрений, а также предотвращать болезни растений.
   - Технологии: IoT, Machine Learning, беспроводные датчики, блокчейн для отслеживания происхождения продуктов.
   - Рынок: Сельскохозяйственная промышленность.

2. **Стартап в области здравоохранения - "Здоровый Образ"**
   - Идея: Создание искусственного интеллекта, который поможет диагностировать заболевания на ранних стадиях путем анализа изображений медицинских тестов и истории болезни пациента.
   - Технологии: Deep Learning, Computer Vision, Natural Language Processing (NLP).
   - Рынок: Медицинская промышленность.

3. **Стартап в области образования - "Умный Школьник"**
   - Идея: Разработка персонализированной платформы для обучения, которая использует Machine Learning для адаптации содержимого и темпа обучения к индивидуальным потребностям уча

### 📌 **SmolAgents**
---
[Библиотека](https://huggingface.co/docs/smolagents/index)


**SmolAgents** – минималистичная open-source-библиотека от Hugging Face для простого создания AI-агентов с помощью минимального количества кода. 

Главная идея – поддержать концепцию «код-агентов»: вместо генерации описания следующего действия в виде текста или JSON, агент сразу пишет и исполняет фрагменты Python-кода. Это позволяет максимально использовать возможности моделей при генерации кода и повышает гибкость логики агента.

При этом SmolAgents даёт полный доступ к множеству моделей и инструментов (поиск в интернете, базы знаний и пр.) и поддерживает разные режимы ввода (текст, изображение, аудио и др.) 


Ключевые возможности и сценарии:
- CodeAgent: основной класс агента, который выполняет Python-код. Такой подход снижает количество необходимых LLM-вызовов и улучшает композицию действий.

- Интеграция с инструментами: легко добавлять существующие «тулы» (например, веб-поиск, утилиты для работы с данными). Инструменты можно загружать и делиться через Hugging Face Hub.

- Поддержка моделей: можно использовать локальные трансформеры, или модели из разных провайдеров (OpenAI, Anthropic, HF), а также подключать модели через LiteLLM и другие обвязки.
- Примеры применения: быстрое прототипирование интерактивных агентов (ответы на запросы с последующим поиском или вычислениями), генерация SQL-запросов, ответы на вопросы с использованием веб-данных и т.д. 

In [ ]:
from huggingface_hub import login, InferenceClient

In [ ]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel() 

agent = CodeAgent(tools=[], model=model)

result = agent.run("Calculate the sum of numbers from 1 to 10")
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate the sum of numbers from 1 to 10                                                                       │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  total_sum = sum(range(1, 11))                                                                                    
  final_answer(total_sum)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 55

[Step 1: Duration 5.08 seconds| Input tokens: 2,008 | Output tokens: 63]

55


**Вывод**: 
    
    LangChain, AutoGen, CrewAI и SmolAgents представляют собой современные инструменты для построения интеллектуальных агентов на базе LLM. 
    
    LangChain – универсальный фреймворк с множеством компонентов для гибкого конструирования цепочек и агентов.
    AutoGen фокусируется на масштабируемой многопроцессной архитектуре с обширными возможностями мониторинга и распределения
    CrewAI предлагает подход «команда агентов» с ролевым распределением задач
    SmolAgents отличается компактностью и использованием «кодагентов» для максимальной простоты и эффективности .
    
    
На семинаре мы попробуем каждую и решим, где какой фреймворк будем использовать